In [ ]:
# Check for the logged in user_id,Add roles BigQuery Job admin,Data Catalog Admin,Policy Tag Admin,Fine-Grained Reader
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       320177532135-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [ ]:
#Set Project
PROJECT_ID='layodemo' # Change to your projectid
BUCKET='bq-demo-layo' # Change to your bucket name
!gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
#Create new bucket for this demo
#US(multiregion)
!gsutil mb gs://$BUCKET

Creating gs://bq-demo-layo/...


In [ ]:
#Copy all 4 files from Shared google drive to the created Bucket
#   https://drive.google.com/corp/drive/u/0/folders/1-LXzBomeYqEepFd32tcy9R6JLoTwU0_5 

In [ ]:
#Create a dataset called "bq_demo" in BigQuery  
!bq --location=US  mk --dataset bq_demo

Dataset 'layodemo:bq_demo' successfully created.


In [ ]:
#Load 150 M/30.8 GB data

In [ ]:
# Load the data into that dataset bq_demo using CLI
!bq load --autodetect=true --source_format=CSV bq_demo.diabetic_data_cli_150m gs://bq-demo-layo/diabetic_data_150m.csv

Waiting on bqjob_r36fb25343c77792a_0000017fa8692fc9_1 ... (47s) Current status: DONE   


In [ ]:
#Show how to load using Cloud console to load the file from GCS to table "diabetic_data_cc_150m".Refer document for steps

In [ ]:
# Load the data into that dataset bq_demo using Python Client
import csv
from google.cloud import bigquery    

client = bigquery.Client(project="layodemo")  # Change the project name
uri = 'gs://bq-demo-layo/diabetic_data_150m.csv'
dataset_id = 'bq_demo'
table_id = 'bq_demo.diabetic_data_clib_150m'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.autodetect = True
job_config.skip_leading_rows=1

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

print("Load is complete")

Load is complete


In [ ]:
#Call REST API using POST method
!curl --request POST \
  https://bigquery.googleapis.com/bigquery/v2/projects/layodemo/jobs \
  --header 'Authorization:Bearer '$(gcloud auth application-default print-access-token) \
  --header 'Accept: application/json' \
  --header 'Content-Type: application/json' \
  --data '{"configuration":{"load":{"sourceUris":["gs://bq-demo-layo/diabetic_data_150m.csv"],"destinationTable":{"projectId":"layodemo","datasetId":"bq_demo","tableId":"diabetic_data_api_150m"},"autodetect":true},"jobType":"LOAD"}}' \
  --compressed

{
  "kind": "bigquery#job",
  "etag": "MURn+gbjZZYpSzC20sfQbA==",
  "id": "layodemo:US.job_QyB50mL1U8UuFQXQvFdKKw4OimK7",
  "selfLink": "https://bigquery.googleapis.com/bigquery/v2/projects/layodemo/jobs/job_QyB50mL1U8UuFQXQvFdKKw4OimK7?location=US",
  "user_email": "320177532135-compute@developer.gserviceaccount.com",
  "configuration": {
    "load": {
      "sourceUris": [
        "gs://bq-demo-layo/diabetic_data_150m.csv"
      ],
      "destinationTable": {
        "projectId": "layodemo",
        "datasetId": "bq_demo",
        "tableId": "diabetic_data_api_150m"
      },
      "autodetect": true
    },
    "jobType": "LOAD"
  },
  "jobReference": {
    "projectId": "layodemo",
    "jobId": "job_QyB50mL1U8UuFQXQvFdKKw4OimK7",
    "location": "US"
  },
  "statistics": {
    "creationTime": "1647801038199",
    "startTime": "1647801038309"
  },
  "status": {
    "state": "RUNNING"
  }
}


In [ ]:
PROJECT_ID='layodemo' # Update ypur project id
JOB_ID="job_QyB50mL1U8UuFQXQvFdKKw4OimK7" # Copy Job_ID form abo›ve output

var = !curl -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  https://www.googleapis.com/bigquery/v2/projects/$PROJECT_ID/jobs/$JOB_ID

var1 = var[-4:]
str1 = ''.join(var1)

print(str1.replace("'", "").replace("}}", "}").replace(" ", ""))

"status":{"state":"DONE"}


In [ ]:
#PARTITION AND CLUSTERING 

In [ ]:
#Load data into table diabetic_data_base
!bq load --autodetect=true --source_format=CSV bq_demo.diabetic_data_base gs://bq-demo-layo/diabetic_data_150m.csv

Waiting on bqjob_r29875855838572e3_0000017fa89b53a5_1 ... (33s) Current status: DONE   


In [ ]:
# Create Schema file
!bq show --format=prettyjson layodemo:bq_demo.diabetic_data_base | jq ".schema.fields" > table.json
print ("Complete")

Complete


In [ ]:
#Create partition by date
!bq mk --table \
  --use_legacy_sql=false \
  --time_partitioning_field date \
  --time_partitioning_type DAY \
  --require_partition_filter=TRUE \
  --schema table.json \
    layodemo:bq_demo.diabetic_data_partition

Table 'layodemo:bq_demo.diabetic_data_partition' successfully created.


In [ ]:
#Load partitioned table with 3 days data
!bq load --autodetect=true --source_format=CSV bq_demo.diabetic_data_partition gs://bq-demo-layo/diabetic_data_150m.csv

Waiting on bqjob_r59490bbe1f7c580f_0000017fa8a25ddf_1 ... (64s) Current status: DONE   


In [ ]:
#Create Clustering table on date and medical_specialty
!bq mk --table \
  --use_legacy_sql=false \
  --clustering_fields date,medical_specialty \
  --schema table.json \
    layodemo:bq_demo.diabetic_data_clustered

Table 'layodemo:bq_demo.diabetic_data_clustered' successfully created.


In [ ]:
#Load data into clustered table
!bq load --autodetect=true --source_format=CSV bq_demo.diabetic_data_clustered gs://bq-demo-layo/diabetic_data_150m.csv

Waiting on bqjob_r53e5f4ce803a2d11_0000017fa8a4e24b_1 ... (47s) Current status: DONE   


In [ ]:
#Create Clustering table only on medical_specialty
!bq mk --table \
  --use_legacy_sql=false \
  --clustering_fields medical_specialty \
  --schema table.json \
    layodemo:bq_demo.diabetic_data_clus

Table 'layodemo:bq_demo.diabetic_data_clus' successfully created.


In [ ]:
#Load data into clustered table
!bq load --autodetect=true --source_format=CSV bq_demo.diabetic_data_clus gs://bq-demo-layo/diabetic_data_150m.csv

Waiting on bqjob_r5c788b27606216f1_0000017fa8a5dcbb_1 ... (47s) Current status: DONE   


In [ ]:
#Create Parttitioned Clustered table
!bq mk --table \
  --use_legacy_sql=false \
  --time_partitioning_field date \
  --time_partitioning_type DAY \
  --require_partition_filter=TRUE \
  --clustering_fields date,medical_specialty\
  --schema table.json \
    layodemo:bq_demo.diabetic_data_partition_clustered

Table 'layodemo:bq_demo.diabetic_data_partition_clustered' successfully created.


In [ ]:
#Load data into partiitoned clustered table
!bq load --autodetect=true --source_format=CSV bq_demo.diabetic_data_partition_clustered gs://bq-demo-layo/diabetic_data_150m.csv

Waiting on bqjob_r116d2b024f767bd7_0000017fa8a84862_1 ... (47s) Current status: DONE   


In [ ]:
#Run below listed Queries in BigQuery to demonstrate the importance of partitioning and clustering.Please refer video for the same at

SELECT * FROM `layodemo.bq_demo.diabetic_data` where date='2022-02-12' and medical_specialty='InternalMedicine'

SELECT * FROM `layodemo.bq_demo.diabetic_data_partition` where date='2022-02-12' and medical_specialty='InternalMedicine'

SELECT * FROM `layodemo.bq_demo.diabetic_data_clustered` where date='2022-02-12' and medical_specialty='InternalMedicine'

SELECT * FROM `layodemo.bq_demo.diabetic_data_clus` where date='2022-02-12' and medical_specialty='InternalMedicine'

SELECT * FROM `layodemo.bq_demo.diabetic_data_partition_clustered` where date='2022-02-12' and medical_specialty='InternalMedicine'


In [ ]:
#ACCESS POLICY-REFER DOCUMENT

In [ ]:
#NESTED JSON

In [ ]:
#Load nested json to BQ using cli
!bq load --autodetect=true --source_format=NEWLINE_DELIMITED_JSON bq_demo.github_nested_json gs://bq-demo-layo/NestedJson/github_nested.json

Waiting on bqjob_r1ee0f875bdfa14ff_0000017fa92eb3e9_1 ... (33s) Current status: DONE   


In [ ]:
#Follow document on how to read nested columns from github_nested_json

In [ ]:
#BigQuery Table CLONE

In [ ]:
!bq cp --clone bq_demo.github_nested_json bq_demo.github_nested_json_clone

Waiting on bqjob_r4017c1c84a66a1f5_0000017fa957ea1d_1 ... (0s) Current status: DONE   
Table 'layodemo:bq_demo.github_nested_json' successfully cloned to 'layodemo:bq_demo.github_nested_json_clone'


In [ ]:
#Time travel

In [ ]:
#Load data into clustered table
!bq load --autodetect=true --source_format=CSV bq_demo.timetravel_ssn gs://bq-demo-layo/patient_SSN.csv

Waiting on bqjob_r172130e3c2045837_0000017fa95c1c1f_1 ... (1s) Current status: DONE   


In [ ]:
#TimeTravel in BigQuery

In [ ]:
!bq cp bq_demo.timetravel_ssn@-540000 bq_demo.timetravel_ssn_restored

Waiting on bqjob_r46f58f51efb4d02b_0000017fa9652887_1 ... (0s) Current status: DONE   
Table 'layodemo:bq_demo.timetravel_ssn@-540000' successfully copied to 'layodemo:bq_demo.timetravel_ssn_restored'


In [ ]:
#THANK YOU